<a href="https://colab.research.google.com/github/saadkhi/GSoC-2025-Task/blob/main/Specific_Test_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth, drive
import gdown
import os

# Authentication for Google Drive
auth.authenticate_user()

# Mount Google Drive
drive.mount('/content/drive')

# Folder ID from the shared link
folder_id = "1-91y1fQHanXfzx5WUy0qLroflfgVnk_L"

# Destination in your Google Drive
destination = "/content/drive/My Drive/data"

# Create the destination folder if it doesn't exist
os.makedirs(destination, exist_ok=True)

# Use gdown to download the folder
gdown.download_folder(f"https://drive.google.com/drive/folders/{folder_id}", output=destination, quiet=False, use_cookies=False)

print("Download complete! Files saved in:", destination)



PDF_PATH_1 = "/content/drive/My Drive/data/Ezcaray - Vozes.pdf"  # Adjust if needed
PDF_PATH_2 = "/content/drive/My Drive/data/Constituciones sinodales Calahorra 1602.pdf"  # Adjust if needed
IMAGE_DIR = "/content/drive/My Drive/data/images"  # Create images in 'data' folder
os.makedirs(IMAGE_DIR, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Retrieving folder contents


Processing file 1-JaE5tNwyNHGDG60GHVihshifrVGgRKj Ezcaray - Vozes.pdf
Processing file 1-7pV-zFu6cVuty3wRhjMTN3XbpBhQzHC Mendo - Principe perfecto.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1-JaE5tNwyNHGDG60GHVihshifrVGgRKj
To: /content/drive/My Drive/data/Ezcaray - Vozes.pdf
100%|██████████| 3.44M/3.44M [00:00<00:00, 44.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-7pV-zFu6cVuty3wRhjMTN3XbpBhQzHC
To: /content/drive/My Drive/data/Mendo - Principe perfecto.pdf
100%|██████████| 2.37M/2.37M [00:00<00:00, 22.4MB/s]

Download complete! Files saved in: /content/drive/My Drive/data



Download completed


Task 1

In [2]:
!pip install pdf2image
!apt-get install -y poppler-utils

import tensorflow as tf
print(tf.__version__)

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pdf2image import convert_from_path
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score, f1_score

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
2.18.0


In [ ]:
# Define paths
pdf_folder_path = "/content/drive/My Drive/data/Ezcaray - Vozes.pdf"
output_image_folder = "/content/pdf_images"

# Create directories if they don't exist
if not os.path.exists(output_image_folder):
    os.makedirs(output_image_folder)

# Function to convert PDF to images
def pdf_to_images(pdf_path, output_folder):
    images = convert_from_path(pdf_path)
    for i, image in enumerate(images):
        image.save(os.path.join(output_folder, f"page_{i}.jpg"), "JPEG")

# Convert PDF to images
pdf_to_images(pdf_folder_path, output_image_folder)

# Function to create simple masks (replace with your actual mask creation)
def create_masks(image_paths):
    masks = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        mask = np.zeros_like(img[:, :, 0], dtype=np.uint8)
        mask[100:img.shape[0]-100, 100:img.shape[1]-100] = 255
        masks.append(mask)
    return masks

# Load images and masks
image_paths = [os.path.join(output_image_folder, f) for f in os.listdir(output_image_folder) if f.endswith(".jpg")]
images = [cv2.imread(path) for path in image_paths]
masks = create_masks(image_paths)

# Normalize images
images = [img / 255.0 for img in images]

# Resize images and masks
resized_images = [cv2.resize(img, (256, 256)) for img in images]
resized_masks = [cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST) / 255.0 for mask in masks]

# Convert to numpy arrays
X = np.array(resized_images)
y = np.array(resized_masks)[:, :, :, np.newaxis]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# U-Net model enhanced with attention
def build_unet(input_shape=(256, 256, 3)):
    inputs = layers.Input(input_shape)
    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    # ... (Add more encoder layers)
    # Bottleneck
    conv_mid = layers.Conv2D(256, 3, activation='relu', padding='same')(pool1)
    conv_mid = layers.Conv2D(256, 3, activation='relu', padding='same')(conv_mid)
    # Decoder
    up7 = layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv_mid)
    merge7 = layers.concatenate([up7, conv1], axis=3)
    conv7 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv7)
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv7)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Compile the model
model = build_unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with ImageDataGenerator
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          steps_per_epoch=int(len(X_train) / 32),  # Corrected line
          epochs=10,
          validation_data=(X_test, y_test))

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(np.uint8)

# Calculate IoU and F1-score
iou = jaccard_score(y_test.flatten(), y_pred_binary.flatten())
f1 = f1_score(y_test.flatten(), y_pred_binary.flatten())

print(f"Test IoU: {iou}, Test F1-Score: {f1}")

In [ ]:
accuracy = accuracy_score(y_test.flatten(), y_pred_binary.flatten())
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.8068084716796875
